In [2]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda
import requests
import pandas as pd
import numpy as np

# Pair Programming ETL I
Vamos a nutrir los datos de los ataques de los tiburones con información climática de los paises que tenemos. Para eso vamos a usar la API del clima que hemos aprendido en la clase invertida:

En este caso os recomendamos que uséis el producto meteo para obtener la información climática. Para hacer la llamada a la API necesitamos también las coordenadas de los paises que tenemos en el dataset.


Por lo tanto, el objetivo es que saquéis la información del clima de la API para los paises que tenemos. Pero antes de poneros manos a la obra, tenemos muchísimos países y esto puede hacerse eterno. Sacad la información climática solo para los siguientes países, con las siguientes coordenadas:

Requisitos de este ejercicio de pair programming:

- Deberéis meter toda la información climática en un único dataframe.
- Deberéis hacer la llamada a la API de una sola vez. Es decir, tendréis que iterar por la lista de países y sacar la información del clima para cada uno de ellos.
- Al meter toda la información en un único dataframe tendremos que crear una columna que corresponda con el nombre del país.
- Guardar el dataframe obtenido para usarlo en la siguiente sesión de pair programming.

💡 Pistas 💡:
- Crearos un diccionario donde:
    -   Las keys sean los nombres de los paises
    -   Los values sean las coordenadas de los paises en formato lista.

- Iterar por el diccionario. Dentro del for haremos la llamada a la API como hemos aprendido en las clases invertidas. Recordamos que para iterar por el diccionario tenemos que seguir la siguiente estructura:
for key, value in diccionario.items():
    print(key, value)

- Crearnos un dataframe vacío donde iremos concatenando los datos obtenidos de la API cada país.
Guardad el csv ya que mañana lo necesitaremos para hacer el ejercicio de pair programming.

In [3]:
# Creamos un diccionario con la latitud y longitud de los paises:                         
                        # Pais : [Latitud, Longitud]
paises_coordenadas = {"usa": [39.7837304, -100.445882], 
                        "australia": [-24.7761086, 134.755], 
                        "south africa": [-28.8166236, 24.991639], 
                        "new zealand": [-41.5000831, 172.8344077], 
                        "papua new guinea": [-5.6816069, 144.2489081]}

In [4]:
def llamada_API(dicc, producto):
#Iteramos por el diccionario en busca de nuestros valores
    lista_dataframes = []
    for k, v in dicc.items():
        lat = v[0]
        lon = v[1]
    
    # hacemos la llamada  a la API
        url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
        response = requests.get(url=url)
        codigo_estado = response.status_code
        razon_estado = response.reason
    # Ponemos esto para comprobar si todo está correcto.    
        if codigo_estado == 200:
            print('La peticion de se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usuario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        
     # convertimos los resultados en un dataframe:
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
    # Creamos columnas con el nombre del pais, la latitud y la longitud para poder hacer la union.
        df["country"] = k
        df["latitud"] = lat
        df["longitud"] = lon

        # Apendeamos el dataframe en una lista que hemos creado
        lista_dataframes.append(df)
    
    # juntamos todos los dataframes
    data = pd.concat(lista_dataframes, axis= 0, ignore_index= True)
    return data

In [5]:
# Llamamos a la función y creamos el dataframe con el producto meteo y las coordenadas que hemos guardado en el diccionario
df = llamada_API( paises_coordenadas, 'meteo')
df.sample(3)

La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud
105,126,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 20, 'speed': ...",26,-4,11,1015,rain,3,0,20,3,australia,-24.776109,134.755000
95,96,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 80, 'speed': ...",25,-4,11,1012,rain,3,0,80,3,australia,-24.776109,134.755000
164,111,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 325, 'speed':...",24,2,10,1018,none,1,0,335,2,south africa,-28.816624,24.991639


In [6]:
# Comprobamos que se hayan incluido
df.country.value_counts()

usa                 64
australia           64
south africa        64
new zealand         64
papua new guinea    64
Name: country, dtype: int64

In [7]:
df.shape

(320, 19)

In [8]:
# Comprobamos también si hay nulos
df.isnull().sum()

timepoint            0
cloudcover           0
highcloud            0
midcloud             0
lowcloud             0
rh_profile           0
wind_profile         0
temp2m               0
lifted_index         0
rh2m                 0
msl_pressure         0
prec_type            0
prec_amount          0
snow_depth           0
wind10m.direction    0
wind10m.speed        0
country              0
latitud              0
longitud             0
dtype: int64

In [9]:
# Guardamos el df
df.to_csv("../data/00-datos_clima.csv")

In [10]:
df.to_pickle('../data/00-datos-clima.pkl')